Calibara

In [ ]:
import cv2
import numpy as np
import os
import json

# Parameters
chessboard_size = (8, 6)  # Number of inner corners (columns, rows)
square_size = 2.5  # Set this to the real-world square size (e.g., cm)
save_path = "calib_images"
calibration_file = "stereo_calibration.json"

os.makedirs(save_path, exist_ok=True)
cap_left = cv2.VideoCapture(1)
cap_right = cv2.VideoCapture(2)

img_counter = 0

print("Press 'c' to capture image pair, 'q' to quit.")

while True:
    retL, frameL = cap_left.read()
    retR, frameR = cap_right.read()

    if not retL or not retR:
        print("Camera error.")
        break

    combined = np.hstack((frameL, frameR))
    cv2.imshow("Stereo Capture - Left | Right", combined)

    key = cv2.waitKey(1)
    if key == ord('c'):
        cv2.imwrite(f"{save_path}/left_{img_counter}.png", frameL)
        cv2.imwrite(f"{save_path}/right_{img_counter}.png", frameR)
        print(f"Saved image pair {img_counter}")
        img_counter += 1
    elif key == ord('q'):
        break

cap_left.release()
cap_right.release()
cv2.destroyAllWindows()


In [ ]:
# Prepare object points
import numpy as np
import cv2
import os
import json
chessboard_size = (8, 6)  # Number of inner corners (columns, rows)
square_size = 2.5  # Set this to the real-world square size (e.g., cm)
save_path = "calib_images"
calibration_file = "stereo_calibration.json"


objp = np.zeros((chessboard_size[1]*chessboard_size[0], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
objp *= square_size

objpoints = []
imgpointsL = []
imgpointsR = []

# Load image pairs
num_pairs = len([f for f in os.listdir(save_path) if f.startswith("left")])
for i in range(num_pairs):
    imgL = cv2.imread(f"{save_path}/left_{i}.png")
    imgR = cv2.imread(f"{save_path}/right_{i}.png")

    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

    retL, cornersL = cv2.findChessboardCorners(grayL, chessboard_size, None)
    retR, cornersR = cv2.findChessboardCorners(grayR, chessboard_size, None)

    if retL and retR:
        objpoints.append(objp)
        imgpointsL.append(cornersL)
        imgpointsR.append(cornersR)

print(f"Used {len(objpoints)} valid pairs for calibration.")


In [ ]:
# Calibrate individual cameras
retL, mtxL, distL, _, _ = cv2.calibrateCamera(objpoints, imgpointsL, grayL.shape[::-1], None, None)
retR, mtxR, distR, _, _ = cv2.calibrateCamera(objpoints, imgpointsR, grayR.shape[::-1], None, None)

# Stereo calibration
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT, 30, 1e-6)
flags = 0  # Can experiment with cv2.CALIB_FIX_INTRINSIC, etc.
retS, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
    objpoints, imgpointsL, imgpointsR,
    mtxL, distL, mtxR, distR, grayL.shape[::-1],
    criteria=criteria, flags=flags
)

# Stereo rectification
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
    mtxL, distL, mtxR, distR, grayL.shape[::-1], R, T
)

# Save calibration to JSON
calibration_data = {
    'mtxL': mtxL.tolist(),
    'distL': distL.tolist(),
    'mtxR': mtxR.tolist(),
    'distR': distR.tolist(),
    'R': R.tolist(),
    'T': T.tolist(),
    'R1': R1.tolist(),
    'R2': R2.tolist(),
    'P1': P1.tolist(),
    'P2': P2.tolist(),
    'Q': Q.tolist()
}

with open(calibration_file, "w") as f:
    json.dump(calibration_data, f, indent=4)

print("Calibration complete and saved to JSON.")


In [ ]:
# Load stereo calibration
with open("stereo_calibration.json", "r") as f:
    calib = json.load(f)

# Convert lists back to NumPy arrays
mtxL = np.array(calib['mtxL'])
distL = np.array(calib['distL'])
mtxR = np.array(calib['mtxR'])
distR = np.array(calib['distR'])
R = np.array(calib['R'])
T = np.array(calib['T'])
R1 = np.array(calib['R1'])
R2 = np.array(calib['R2'])
P1 = np.array(calib['P1'])
P2 = np.array(calib['P2'])
Q = np.array(calib['Q'])


print(mtxL, distL, mtxR, distR, R, T, R1, R2, P1, P2, Q)


In [ ]:
# Start cameras
capL = cv2.VideoCapture(1)
print('done1')
capR = cv2.VideoCapture(2)
print('done2')

if not capL.isOpened():
        print("Left camera not opened.")
if not capR.isOpened():
        print("Right camera not opened.")

# Set same resolution
width, height = 640, 480
capL.set(cv2.CAP_PROP_FRAME_WIDTH, width)
print('done 3')
capL.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
print('done 4')
capR.set(cv2.CAP_PROP_FRAME_WIDTH, width)
print('done 4')
capR.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
print('done 4')

# Generate undistortion and rectification maps
mapLx, mapLy = cv2.initUndistortRectifyMap(mtxL, distL, R1, P1, (width, height), cv2.CV_32FC1)
print('done 5')
mapRx, mapRy = cv2.initUndistortRectifyMap(mtxR, distR, R2, P2, (width, height), cv2.CV_32FC1)
print('done 6, end')



In [ ]:
# Start cameras
capL = cv2.VideoCapture(1)
print('done1')
capR = cv2.VideoCapture(2)
print('done2')
# Stereo matcher
stereo = cv2.StereoBM_create(numDisparities=64, blockSize=15)  # Adjust if needed

print("Press 'q' to quit.")
while True:
    retL, frameL = capL.read()
    retR, frameR = capR.read()
    if not retL or not retR:
        break

    # retL, frameL = capL.read()
    # retR, frameR = capR.read()
    if retL and retR:
        cv2.imshow("Left", frameL)
        cv2.imshow("Right", frameR)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    else:
        print("Camera read failed.")


    # Rectify and convert to grayscale
    rectL = cv2.remap(frameL, mapLx, mapLy, cv2.INTER_LINEAR)
    rectR = cv2.remap(frameR, mapRx, mapRy, cv2.INTER_LINEAR)
    grayL = cv2.cvtColor(rectL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(rectR, cv2.COLOR_BGR2GRAY)

    # Compute disparity map
    disparity = stereo.compute(grayL, grayR).astype(np.float32) / 16.0

    # Compute depth map using Q matrix
    depth_map = cv2.reprojectImageTo3D(disparity, Q)
    distances = depth_map[:, :, 2]

    # Mask out bad values
    valid = (disparity > 0) & (distances < 10000)

    # Minimum distance from valid points
    if np.any(valid):
        min_distance = np.min(distances[valid])
        text = f"Min distance: {min_distance:.2f} cm"
        if min_distance < 50:
            text += " - WARNING: TOO CLOSE!"
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
    else:
        text = "No valid depth"
        color = (0, 255, 255)

    # Display
    disp_vis = cv2.normalize(disparity, None, 0, 255, cv2.NORM_MINMAX)
    disp_vis = cv2.applyColorMap(disp_vis.astype(np.uint8), cv2.COLORMAP_JET)

    cv2.putText(disp_vis, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    cv2.imshow("Disparity Map", disp_vis)

    if cv2.waitKey(1) == ord('q'):
        break

capL.release()
capR.release()
cv2.destroyAllWindows()


In [ ]:
# Stereo SGBM matcher settings
min_disp = 0
num_disp = 64  # Must be divisible by 16
block_size = 5
stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 3 * block_size**2,
    P2=32 * 3 * block_size**2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

print("Press 'q' to quit.")
while True:
    retL, frameL = capL.read()
    retR, frameR = capR.read()
    if not retL or not retR:
        break

    # Rectify and convert to grayscale
    rectL = cv2.remap(frameL, mapLx, mapLy, cv2.INTER_LINEAR)
    rectR = cv2.remap(frameR, mapRx, mapRy, cv2.INTER_LINEAR)
    grayL = cv2.cvtColor(rectL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(rectR, cv2.COLOR_BGR2GRAY)

    # Compute disparity
    disparity = stereo.compute(grayL, grayR).astype(np.float32) / 16.0

    # Reproject to 3D
    depth_map = cv2.reprojectImageTo3D(disparity, Q)
    distances = depth_map[:, :, 2]

    # Filter out bad values
    valid = (disparity > min_disp) & (distances < 10000)

    if np.any(valid):
        min_distance = np.min(distances[valid])
        text = f"Min distance: {min_distance:.2f} cm"
        if min_distance < 50:
            text += " - WARNING: TOO CLOSE!"
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
    else:
        text = "No valid depth"
        color = (0, 255, 255)

    # Visualize disparity
    disp_vis = cv2.normalize(disparity, None, 0, 255, cv2.NORM_MINMAX)
    disp_vis = cv2.applyColorMap(disp_vis.astype(np.uint8), cv2.COLORMAP_JET)

    cv2.putText(disp_vis, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    cv2.imshow("Disparity Map (SGBM)", disp_vis)

    if cv2.waitKey(1) == ord('q'):
        break

capL.release()
capR.release()
cv2.destroyAllWindows()
